In [1]:
import pandas as pd
import numpy as np
import altair as alt

In [2]:
county=pd.read_csv('US counties - education vs per capita personal income - results-20221227-213216.csv')

In [3]:
county

,county_FIPS,state,county,per_capita_personal_income_2019,per_capita_personal_income_2020,per_capita_personal_income_2021,associate_degree_numbers_2016_2020,bachelor_degree_numbers_2016_2020,associate_degree_percentage_2016_2020,bachelor_degree_percentage_2015_2019
0,51013,VA,"Arlington, VA",97629,100687,107603,19573,132394,11.21,75.84
1,35028,NM,"Los Alamos, NM",72366,75949,81306,2766,9098,20.54,67.56
2,8013,CO,"Boulder, CO",79698,83173,89593,45834,135876,21.24,62.97
3,24027,MD,"Howard, MD",78013,82041,86380,42538,136792,19.49,62.67
4,36061,NY,"New York, NY",173525,175327,195543,167960,777483,13.43,62.18
...,...,...,...,...,...,...,...,...,...,...
3001,13165,GA,"Jenkins, GA",30946,34673,36886,1561,315,25.87,5.22
3002,28055,MS,"Issaquena, MS",24805,32026,34832,271,34,27.07,3.40
3003,2158,AK,"Kusilvak Census Area, AK",31050,32971,36753,854,132,21.11,3.26
3004,48261,TX,"Kenedy, TX",43786,45009,47141,16,4,5.93,1.48


In [4]:
county=county[['state', 'county', 'per_capita_personal_income_2021','bachelor_degree_percentage_2015_2019']]

In [5]:
county['log_income']=np.log1p(county['per_capita_personal_income_2021'])

C:\Users\zyk54\AppData\Local\Temp\ipykernel_11940\3700373682.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  county['log_income']=np.log1p(county['per_capita_personal_income_2021'])


In [6]:
county

,state,county,per_capita_personal_income_2021,bachelor_degree_percentage_2015_2019,log_income
0,VA,"Arlington, VA",107603,75.84,11.586213
1,NM,"Los Alamos, NM",81306,67.56,11.305987
2,CO,"Boulder, CO",89593,62.97,11.403044
3,MD,"Howard, MD",86380,62.67,11.366523
4,NY,"New York, NY",195543,62.18,12.183541
...,...,...,...,...,...
3001,GA,"Jenkins, GA",36886,5.22,10.515614
3002,MS,"Issaquena, MS",34832,3.40,10.458320
3003,AK,"Kusilvak Census Area, AK",36753,3.26,10.512002
3004,TX,"Kenedy, TX",47141,1.48,10.760920


In [7]:
select_interval = alt.selection_interval()

heatmap = alt.Chart(county).mark_rect().encode(
    x=alt.X('bachelor_degree_percentage_2015_2019:Q', bin=alt.Bin(maxbins=20), title='Bachelor Degree Percentage'),
    y=alt.Y('log_income:Q', bin=alt.Bin(maxbins=20), title='Log of Per Capita Personal Income 2021'),
    color=alt.Color('count():Q',
                    scale=alt.Scale(type='threshold',
                                    domain=[1, 2, 5, 10, 20, 50, 100, 200, 500],
                                    range=['#f7fcf5', '#e5f5e0', '#c7e9c0', '#a1d99b', '#74c476', '#41ab5d', '#238b45', '#006d2c', '#00441b']),
                    legend=alt.Legend(title='Number of Counties'))
).properties(
    title='Average Personal Income vs. Bachelor Degree Percentage by County in the US (2015-2019)',
    width=500,
    height=300
).add_selection(
    select_interval
)

bar1 = alt.Chart(county).mark_bar().encode(
    x=alt.X('per_capita_personal_income_2021:Q', title='Per Capita Personal Income 2021'),
    y=alt.Y('county:N', sort='-x', title='County'),
    color=alt.Color('bachelor_degree_percentage_2015_2019', scale=alt.Scale(scheme='blues'), 
                legend=alt.Legend(title='Bachelor Degree Percentage')),
    tooltip=['county', 'per_capita_personal_income_2021', 'bachelor_degree_percentage_2015_2019']
).transform_filter(
    select_interval
).transform_window(
    rank='rank(per_capita_personal_income_2021)',
    sort=[alt.SortField('per_capita_personal_income_2021', order='descending')]
).transform_filter(
    (alt.datum.rank <= 10)
).properties(
    title='Top 10 Counties within Selected Bachelor Degree Percentage and Personal Income range',
    width=500,
    height=300
)

chart = heatmap & bar1


c:\Users\zyk54\anaconda3\Lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


In [8]:
chart

alt.VConcatChart(...)